In [24]:
import pandas as pd

In [25]:
## In alltrails directory: https://www.alltrails.com/directory/trails/A/1 
## Grab urls to all the trails 

In [135]:
## First item in directory starts, within class "directory-content", ends with class "directory-pages"
## each item starts with item tag 
## link in ahref tag 

## example: <a title="Agua Caliente, Sierra Caliente and Monument Peak" 
## href="/trail/us/california/ed-r-levin-county-park-trail"

In [137]:
## filter content to start after the directory-content tag and before the directory-pages tag
## get filtered results begins with "<a title=""
## ends with "</li>\n      <li>\n"
## expression: (?<=title).*?(?=li>)

In [226]:
## function to loop get the # of pages for each letter of the alphabet

## input -- alphabet to get

def getPages(letter):
    
    ## construct the url 
    url = "https://www.alltrails.com/directory/trails/{}/1".format(letter)
    
    ## make the request 
    r = requests.get(url)
    text = r.text
    
    ## get the targeted text block 
    pages_text = text.split(sep ="directory-pages")[-1]
    pages_text = pages_text.split(sep = "#content")[0]
    
    ## split by attributes and get results 
    page_numbers  = pages_text.split("a href")[-1].split("\"")[1][-1]
    return(page_numbers)


yo = getPages('X')

In [229]:
## function to make request for each letter of the alphabet

## input -- letter of the alphabet | number of pages for specific letter 

def getTrails(letter):
    
    ## initiate empty list 
    alphabet_list = []
    
    ## get the number of pages for the letter 
    max_pages = getPages(letter)
    
    ## loop through each specific page
    counter = 0
    
    while counter < int(max_pages):
        counter += 1 
        
        ## construct url 
        url = "https://www.alltrails.com/directory/trails/{}/{}".format(letter, counter)
        r = requests.get(url)
        text = r.text
        
        ## extract section of trails 
        text = text.split(sep ="directory-content")[-1]
        text = text.split(sep = "directory-pages")[0]
        
        ## regex to find list of sections with trail info 
        trails_phase = re.findall("title(.*?)class", string=text)
        
        ## loop through each secrion to obtain trail name and corresponding url 
        for value in trails_phase:
            
            ## split value by quotes 
            a_list = value.split(sep = "href")
            if (len(a_list) == 2):
                url = a_list[1]
                trail = a_list[0]
                alphabet_list.append([trail[2:], url[2:]])
            else:
                trail = a_list[0]
                alphabet_list.append([trail[2:], " "])
    ## return the list of trails for the corresponding alphabet        
    return(alphabet_list)
        

In [230]:
## initialize the scraper 
import string
import pandas as pd 
import re
import requests

## initialize the final trails list 
alphabet_list = list(string.ascii_uppercase)
## alphabet_list = 'C'
final_trails_list = []

## loop through each letter of alphabet 
for letter in alphabet_list: 
    letter_trail = getTrails(letter)
    print("Finished letter {}".format(letter))
    final_trails_list = final_trails_list + letter_trail

Finished letter A
Finished letter B
Finished letter C
Finished letter D
Finished letter E
Finished letter F
Finished letter G
Finished letter H
Finished letter I
Finished letter J
Finished letter K
Finished letter L
Finished letter M
Finished letter N
Finished letter O
Finished letter P
Finished letter Q
Finished letter R
Finished letter S
Finished letter T
Finished letter U
Finished letter V
Finished letter W
Finished letter X
Finished letter Y
Finished letter Z


In [231]:
len(final_trails_list)

70586

In [234]:
trail_df = pd.DataFrame(final_trails_list, columns = ["name", "url"])
trail_df.head()

,name,url
0,"A and C Trails""","/trail/us/virginia/freedom-park-trails"""
1,"A Cabo de San Juan de Guia Cape""",/trail/colombia/magdalena/a-cabo-de-san-juan-d...
2,"A Coruña""","/trail/spain/galicia/a-coruna"""
3,"A Davis Trail""","/trail/us/west-virginia/a-davis-trail"""
4,"A el Peñón de Ronda desde los Quejigales""",/trail/spain/andalusia/a-el-penon-de-ronda-des...


In [235]:
trail_df.to_csv("trails.csv")